# *Salinas*
# Remuestreo con ROS y SMOTE, Primer entrenamiento y guardar los pesos y los modelos

##  Importando Librerias

In [1]:
import tensorflow as tf
from tensorflow import keras
import random
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.multiclass import OneVsRestClassifier
import os
from sklearn.metrics import confusion_matrix, classification_report 
import pandas as pd 
import seaborn as sn 
from collections import Counter

from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras import optimizers
import datetime

## Definiendo  parametros

In [2]:
directorio="../../"
directorioPrueba=directorio+"Datasets/txt/test/"
directorioEntrenamiento=directorio+"Datasets/txt/sampled_train/"
bases=["Salinas"]
epocas=[500]
lotes=[500]
clases=[17]
bandas=[224]
###el metodo vacio indica que  no se aplica sobre muestreo
metodos=["","_SMOTE","_ROS"]

In [3]:
inicio= datetime.datetime.now()
contadorBases=0
for base in bases:
    print("\n\n\n")
    print(">>>>> Base: ",base)
    for metodo in metodos:
        print(">>>>>>>>>> Metodo: ",metodo)
        print("\n\n\n")
        print("Iniciando proceso para la base: ",base," con el metodo de muestreo: ",metodo)
        try:
            ##Definiendo las rutas de los archivos en funcion a su  base
            print("> Cargando datos de prueba: ",(directorioPrueba+base+".txt"))
            datosPrueba=np.loadtxt(directorioPrueba+base+".txt")
            print("> Cargando datos de Entrenamiento: ",(directorioEntrenamiento+base+".txt" ))
            datosEntrenamiento=np.loadtxt(str(directorioEntrenamiento+base+metodo+".txt" ))
            ## Forma de los datos de prueba
            print(datosPrueba.shape)
            print(datosEntrenamiento.shape)

            columnasD=datosEntrenamiento.shape[1]-1
            columnaC=datosEntrenamiento.shape[1]-1
            x1=datosEntrenamiento[:,0:columnasD]
            y1=datosEntrenamiento[:,columnaC]
            x2=datosPrueba[:,0:columnasD]
            y2=datosPrueba[:,columnaC]
            y1=y1.astype(int)
            y2=y2.astype(int)
            print(len(x1),len(x2),len(y1),len(y2))


            print(">> Iniciando el entrenamiento con: ", str(epocas[contadorBases])+" epocas, con  "+str(lotes[contadorBases])+" muestra por lote, "+str(clases[contadorBases])+" numero de clases, "+str(bandas[contadorBases])+" numero de bandas  ")
            tf.keras.backend.clear_session() 
            adam=tf.keras.optimizers.Adam(learning_rate=0.001)
            capa_entrada= Input(shape=(bandas[contadorBases],))
            capa1= Dense(50, activation='relu')(capa_entrada)
            capa2= Dense(40, activation='relu')(capa1)
            capa3= Dense(30, activation='relu')(capa2)
            capa4= Dense(20, activation='relu')(capa3)
            capa_salida= Dense(clases[contadorBases], activation='softmax')(capa4)

            salida = Model(inputs=capa_entrada,outputs=capa_salida)
            c4= Model(inputs=capa_entrada, outputs=capa4)
            c4.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            c3= Model(inputs=capa_entrada, outputs=capa3)
            c3.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            c2= Model(inputs=capa_entrada, outputs=capa2)
            c2.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            c1= Model(inputs=capa_entrada, outputs=capa1)
            c1.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            
            salida.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            snn=salida.fit(x1,y1,batch_size=lotes[contadorBases],epochs=(epocas[contadorBases]),shuffle=True)

            evaluation = salida.evaluate(x2,y2,batch_size=100,verbose=1)#x1,y1,batch_size=100,verbose=1
            print(evaluation)
            print("\n\n")

            print(">>>>>Guardando Modelo:"+directorio+"modelos/modelo"+base+metodo+".h5")
            salida.save(str(directorio+"modelos/modelo"+base+metodo+".h5"))

            print(">>>>>Guardando Pesos:"+directorio+"pesos/pesos"+base+metodo+'.weights.h5')
            salida.save_weights(str(directorio+"pesos/pesos"+base+metodo+'.weights.h5'))

            print(">>>>>Guardando Modelo:"+ directorio+"modelos/modeloC1"+base+metodo+".h5" )
            c1.save(str( directorio+"modelos/modeloC1"+base+metodo+".h5" ))

            print(">>>>>Guardando Pesos:"+directorio+"pesos/pesosC1"+base+metodo+'.weights.h5' )
            c1.save_weights(str( directorio+"pesos/pesosC1"+base+metodo+'.weights.h5' ))

            print(">>>>>Guardando Modelo:"+ directorio+"modelos/modeloC2"+base+metodo+".h5" )
            c2.save(str( directorio+"modelos/modeloC2"+base+metodo+".h5" ))

            print(">>>>>Guardando Pesos:"+directorio+"pesos/pesosC2"+base+metodo+'.weights.h5' )
            c2.save_weights(str( directorio+"pesos/pesosC2"+base+metodo+'.weights.h5' ))

            print(">>>>>Guardando Modelo:"+ directorio+"modelos/modeloC3"+base+metodo+".h5" )
            c3.save(str(directorio+"modelos/modeloC3"+base+metodo+".h5"))

            print(">>>>>Guardando Pesos:"+directorio+"pesos/pesosC3"+base+metodo+'.weights.h5' )
            c3.save_weights(str(directorio+'/pesos/pesosC3'+base+metodo+'.weights.h5'))

            print(">>>>>Guardando Modelo:"+ directorio+"modelos/modeloC4"+base+metodo+".h5" )
            c4.save(str(directorio+"modelos/modeloC4"+base+metodo+".h5"))

            print(">>>>>Guardando Pesos:"+directorio+"pesos/pesosC4"+base+metodo+'.weights.h5' )
            c4.save_weights(str(directorio+'/pesos/pesosC4'+base+metodo+'.weights.h5'))

            salida.summary()
            c1.summary()
            c2.summary()
            c3.summary()
            c4.summary()

            del salida,c1,c2,c3,c4

        except Exception as error:
            print("> Error realizando el proceso para la base: ",base," con el metodo de muestreo: ",metodo)
            print(">>Error: ",error)
    contadorBases=contadorBases+1
fin= datetime.datetime.now()
print("\n\n\n")
print(">>> Proceso terminado en : ",(fin-inicio))





>>>>> Base:  Salinas
>>>>>>>>>> Metodo:  




Iniciando proceso para la base:  Salinas  con el metodo de muestreo:  
> Cargando datos de prueba:  ../../Datasets/txt/test/Salinas.txt
> Cargando datos de Entrenamiento:  ../../Datasets/txt/sampled_train/Salinas.txt
(33332, 225)
(77772, 225)
77772 33332 77772 33332
>> Iniciando el entrenamiento con:  500 epocas, con  500 muestra por lote, 17 numero de clases, 224 numero de bandas  
Epoch 1/500
156/156 [==============================] - 4s 5ms/step - loss: 1.8700 - accuracy: 0.5143
Epoch 2/500
156/156 [==============================] - 1s 5ms/step - loss: 1.2550 - accuracy: 0.6131
Epoch 3/500
156/156 [==============================] - 1s 6ms/step - loss: 1.1240 - accuracy: 0.6229
Epoch 4/500
156/156 [==============================] - 1s 6ms/step - loss: 1.0471 - accuracy: 0.6432
Epoch 5/500
156/156 [==============================] - 1s 6ms/step - loss: 0.9802 - accuracy: 0.6617
Epoch 6/500
156/156 [==============================] - 1s 6